# DBSCAN - visualization

Beata Baczynska

In [0]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.metrics import accuracy_score
from sklearn.cluster import DBSCAN
from sklearn.neighbors import NearestNeighbors

from sklearn.preprocessing import LabelEncoder
from sklearn import preprocessing
from sklearn.model_selection import train_test_split

import seaborn as sns
sns.set()

from ipywidgets import interact
import ipywidgets as widgets

In [2]:
from google.colab import drive
drive.mount("/content/drive")

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [3]:
%cd "/content/drive/My Drive/Colab Notebooks/Omdena"

/content/drive/My Drive/Colab Notebooks/Omdena


In [0]:
df = pd.read_csv("OUTPUT_WBI_exposer_cyclones_v2.csv", sep=";")

In [44]:
df.columns

Index(['SID', 'NAME', 'ISO', 'YEAR', 'BASIN', 'SUB BASIN', 'NATURE',
       'ISO_TIME', 'COORDS', 'TOTAL_HRS', 'DAY_HRS', 'NIGHT_HRS', 'USA_SSHS',
       'WIND_CALC_MEAN', 'PRES_CALC_MEAN', 'STORM_SPD_MEAN', 'STORM_DR_MEAN',
       'V_LAND_KN', '34KN_ASSETS', '64KN_ASSETS', '96KN_ASSETS', 'CPI',
       'TOTAL_DAMAGE_(000$)', 'TOTAL_DEATHS',
       'Air transport, freight (million ton-km)',
       'Arable land (hectares per person)', 'Cereal yield (kg per hectare)',
       'Food production index (2004-2006 = 100)', 'GDP growth (annual %)',
       'GDP per capita (constant 2010 US$)', 'Net flows from UN agencies US$',
       'Life expectancy at birth, total (years)',
       'Mobile cellular subscriptions (per 100 people)',
       'Population density (people per sq. km of land area)',
       'Adjusted savings: education expenditure (% of GNI)',
       'Rural population (% of total population)', 'Income_level_Final',
       'pop_max_34_adj', 'pop_max_50_adj', 'pop_max_64_adj', 'TOTAL_AFFEC

In [46]:
# choosing some numeric features
cols = ['YEAR', 'TOTAL_HRS', 'NIGHT_HRS', 'USA_SSHS', 'WIND_CALC_MEAN', 'PRES_CALC_MEAN', 'STORM_SPD_MEAN', 'STORM_DR_MEAN', 'V_LAND_KN', 'CPI', 'TOTAL_DAMAGE_(000$)', 'TOTAL_DEATHS', 'Air transport, freight (million ton-km)', 'Arable land (hectares per person)', 'Cereal yield (kg per hectare)', 'Food production index (2004-2006 = 100)', 'GDP growth (annual %)', 'GDP per capita (constant 2010 US$)', 'Net flows from UN agencies US$', 'Life expectancy at birth, total (years)', 'Mobile cellular subscriptions (per 100 people)', 'Population density (people per sq. km of land area)', 'Adjusted savings: education expenditure (% of GNI)', 'Rural population (% of total population)', 'pop_max_34_adj', 'pop_max_50_adj', 'pop_max_64_adj']
df = df[cols]
df

,YEAR,TOTAL_HRS,NIGHT_HRS,USA_SSHS,WIND_CALC_MEAN,PRES_CALC_MEAN,STORM_SPD_MEAN,STORM_DR_MEAN,V_LAND_KN,CPI,TOTAL_DAMAGE_(000$),TOTAL_DEATHS,"Air transport, freight (million ton-km)",Arable land (hectares per person),Cereal yield (kg per hectare),Food production index (2004-2006 = 100),GDP growth (annual %),GDP per capita (constant 2010 US$),Net flows from UN agencies US$,"Life expectancy at birth, total (years)",Mobile cellular subscriptions (per 100 people),Population density (people per sq. km of land area),Adjusted savings: education expenditure (% of GNI),Rural population (% of total population),pop_max_34_adj,pop_max_50_adj,pop_max_64_adj
0,1949,0,0,0,39.133333,1000.333333,19.000000,91.444444,99.820,9.327572,NaN,419.0,NaN,0.059615,4173.500000,67.790000,NaN,8607.657082,0.0,67.666098,NaN,258.911917,2.867878,36.728000,1.187145e+07,3.193122e+06,5.339257e+05
1,1950,6,6,-1,76.400000,974.833333,23.000000,37.666667,99.820,9.431937,NaN,509.0,NaN,0.059615,4173.500000,67.790000,NaN,8607.657082,0.0,67.666098,NaN,258.911917,2.867878,36.728000,2.845835e+07,1.061136e+07,2.985054e+06
2,1951,15,15,1,88.500000,974.500000,15.500000,283.500000,132.400,10.175534,56000.0,154.0,NaN,0.100541,868.300000,69.930000,NaN,3796.219401,0.0,64.770000,NaN,152.453647,2.578304,66.231000,1.689243e+06,1.599244e+06,6.550875e+05
3,1951,15,15,2,84.488235,979.549020,3.000000,161.647059,119.200,10.175534,NaN,569.0,NaN,0.180442,996.300000,25.680000,NaN,1059.467412,0.0,61.105000,NaN,91.065000,2.757732,69.703000,4.752653e+06,2.844443e+06,5.896887e+05
4,1952,3,1,1,83.920000,980.600000,10.800000,291.400000,110.691,10.410354,NaN,19.0,NaN,0.180442,996.300000,25.680000,NaN,1059.467412,0.0,61.105000,NaN,91.065000,2.757732,69.703000,5.119853e+06,2.678770e+06,1.505678e+06
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1000,2019,9,9,3,77.166667,971.611111,3.000000,166.055556,151.200,100.000000,NaN,3.0,NaN,0.141861,3825.634771,121.054617,3.921782,478.761938,0.0,66.012095,35.712976,43.326343,2.080000,63.839788,2.339825e+05,0.000000e+00,0.000000e+00
1001,2019,3,2,3,74.571429,974.714286,8.857143,190.857143,151.200,100.000000,2000000.0,603.0,NaN,0.204973,831.865274,147.309625,4.188907,585.650609,0.0,58.508514,51.655359,35.876238,5.380000,64.832034,1.978779e+06,0.000000e+00,0.000000e+00
1002,2019,3,3,4,121.500000,937.000000,10.000000,248.500000,118.800,100.000000,NaN,8.0,NaN,0.082332,1354.663449,108.375163,3.199964,1375.714129,0.0,63.742834,57.220464,432.007513,4.326607,71.300771,NaN,NaN,NaN
1003,2019,0,0,0,75.444444,971.777778,6.777778,239.000000,119.000,100.000000,230000.0,45.0,NaN,0.204973,831.865274,147.309625,4.188907,585.650609,0.0,58.508514,51.655359,35.876238,5.380000,64.832034,3.494627e+06,5.495747e+05,9.133066e+04


In [49]:
df_dict = {}
i = 1
for x in cols:
  df_dict[i] = x
  i+=1

df_dict

{1: 'YEAR',
 2: 'TOTAL_HRS',
 3: 'NIGHT_HRS',
 4: 'USA_SSHS',
 5: 'WIND_CALC_MEAN',
 6: 'PRES_CALC_MEAN',
 7: 'STORM_SPD_MEAN',
 8: 'STORM_DR_MEAN',
 9: 'V_LAND_KN',
 10: 'CPI',
 11: 'TOTAL_DAMAGE_(000$)',
 12: 'TOTAL_DEATHS',
 13: 'Air transport, freight (million ton-km)',
 14: 'Arable land (hectares per person)',
 15: 'Cereal yield (kg per hectare)',
 16: 'Food production index (2004-2006 = 100)',
 17: 'GDP growth (annual %)',
 18: 'GDP per capita (constant 2010 US$)',
 19: 'Net flows from UN agencies US$',
 20: 'Life expectancy at birth, total (years)',
 21: 'Mobile cellular subscriptions (per 100 people)',
 22: 'Population density (people per sq. km of land area)',
 23: 'Adjusted savings: education expenditure (% of GNI)',
 24: 'Rural population (% of total population)',
 25: 'pop_max_34_adj',
 26: 'pop_max_50_adj',
 27: 'pop_max_64_adj'}

In [52]:
#Dropping nan values
df.dropna(inplace = True)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [0]:
colors = ['royalblue', 'maroon', 'forestgreen', 'mediumorchid', 'tan', 'deeppink', 'olive', 
          'goldenrod', 'lightcyan', 'navy', 'gold', 'aliceblue', 'fuchsia',
          'lime', 'yellow', 'sienna', 'black', 'crimson', 'orange']
vectorizer = np.vectorize(lambda x: colors[x % len(colors)])

In [54]:
x = df.values #returns a numpy array
min_max_scaler = preprocessing.MinMaxScaler()
x_scaled = min_max_scaler.fit_transform(x)
df = pd.DataFrame(x_scaled)
df

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26
0,0.000000,0.000000,0.043478,0.4,0.152519,0.000000,0.135709,0.511253,0.653241,0.000000,0.000091,0.000499,0.000161,0.099653,0.177140,0.229523,0.581364,0.012248,0.015946,0.028026,0.000000,0.007790,0.160850,1.000000,0.004425,0.002084,0.004415
1,0.000000,0.954545,0.956522,0.9,0.715449,0.923647,0.197196,0.839368,0.858704,0.000000,0.000019,0.000983,0.000505,0.037496,0.112123,0.000000,0.532399,0.019461,0.025361,0.482370,0.000000,0.097526,0.157686,0.756450,0.015835,0.004257,0.002873
2,0.000000,0.000000,0.043478,0.9,0.653172,0.927518,0.301334,0.791917,0.755280,0.000000,0.000781,0.005673,0.000505,0.037496,0.112123,0.000000,0.532399,0.019461,0.025361,0.482370,0.000000,0.097526,0.157686,0.756450,0.050118,0.034074,0.004279
3,0.020833,0.409091,0.434783,0.6,0.425373,0.961380,0.262581,0.826402,0.608886,0.007851,0.000005,0.000405,0.000735,0.035997,0.102118,0.013051,0.586293,0.020008,0.038694,0.483235,0.000000,0.100420,0.160850,0.749819,0.049230,0.012269,0.010718
4,0.041667,0.818182,0.826087,0.4,0.267222,0.975575,0.155865,0.541030,0.542607,0.014093,0.000184,0.001467,0.000858,0.035286,0.088243,0.008101,0.586881,0.020575,0.033524,0.483531,0.000000,0.103391,0.116357,0.743124,0.028694,0.011350,0.016998
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
530,1.000000,0.954545,0.956522,0.9,0.683835,0.945866,0.176350,0.281771,0.768390,1.000000,0.004294,0.000036,0.025366,0.056844,0.403792,0.790879,0.479651,0.183006,0.055935,0.773016,0.641511,0.052031,0.340059,0.147840,0.008365,0.005041,0.010194
531,1.000000,0.136364,0.086957,0.5,0.425373,0.965634,0.186963,0.796944,0.868900,1.000000,0.002440,0.000079,0.019446,0.011703,0.397090,0.731646,0.612692,0.051019,0.110257,0.669183,0.850115,0.293652,0.088987,0.576771,0.125460,0.014167,0.010527
532,1.000000,0.409091,0.434783,0.5,0.297042,0.978361,0.207448,0.753644,0.338674,1.000000,0.006200,0.000318,0.062904,0.033659,0.319830,0.974692,0.631076,0.034548,0.555072,0.620159,0.585674,0.373911,0.181810,0.742892,0.104308,0.087534,0.076698
533,1.000000,0.136364,0.086957,0.4,0.036780,0.993054,0.158922,0.754748,0.010925,1.000000,0.000138,0.000130,0.011198,0.018723,0.645724,0.924316,0.639640,0.032110,0.132328,0.778582,0.991556,0.252748,0.295821,0.718523,0.118331,0.015943,0.012151


In [55]:
m = DBSCAN(eps=0.5, min_samples=4)
m.fit(df)

clusters = m.labels_
len(np.unique(clusters))

15

In [0]:
def scatt(x, y):
  plt.scatter(df[x], df[y], c=vectorizer(clusters))
  plt.xlabel(df_dict[x])
  plt.ylabel(df_dict[y])
  plt.show()

In [68]:
widget_input_x = widgets.IntSlider(min=min(df_dict.keys()),max=max(df_dict.keys())-1,step=1,value=1)
widget_input_y = widgets.IntSlider(min=min(df_dict.keys()),max=max(df_dict.keys())-1,step=1,value=1)
interact(scatt, x=widget_input_x, y=widget_input_y)

interactive(children=(IntSlider(value=1, description='x', max=26, min=1), IntSlider(value=1, description='y', …

<function __main__.scatt>